____

# Ho Thi Hai Yen

## Python Intermediate Programming Assignment 2

___

# 1. Import required packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

# 2. Data import